In [2]:
import gc, os
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
sys.path.append(f'/home/{os.environ.get("USER")}/PythonLibrary')
import lgbextension as ex
import lightgbm as lgb
from multiprocessing import cpu_count, Pool
from glob import glob
import utils, utils_cat


In [3]:
ls -lh ../output/*X*

-rw-rw-r-- 1 Kazuki Kazuki 13K Jul 22 01:34 ../output/722-1_X.csv.gz
-rw-rw-r-- 1 Kazuki Kazuki 14K Jul 22 03:54 ../output/722-2_X.csv.gz
-rw-rw-r-- 1 Kazuki Kazuki 17K Jul 24 21:28 ../output/725-1_X.csv.gz
-rw-rw-r-- 1 Kazuki Kazuki 15K Jul 25 05:24 ../output/725-2_X.csv.gz
-rw-rw-r-- 1 Kazuki Kazuki 17K Jul 25 12:16 ../output/725-3_X.csv.gz
-rw-rw-r-- 1 Kazuki Kazuki 13K Jul 25 15:51 ../output/725-4_X.csv.gz


In [4]:
df_lb802 = pd.read_csv('../output/722-2_X.csv.gz')
df_lb800 = pd.read_csv('../output/725-1_X.csv.gz')
df_lb804 = pd.read_csv('../output/725-2_X.csv.gz')
df_lb803 = pd.read_csv('../output/725-4_X.csv.gz')

In [5]:
sub_lb802 = pd.read_csv('../output/722-2.csv.gz')
sub_lb800 = pd.read_csv('../output/725-1.csv.gz')
sub_lb804 = pd.read_csv('../output/725-2.csv.gz')

In [6]:
sub_lb804.TARGET.corr(sub_lb800.TARGET)

0.9889504335682618

In [7]:
df_lb800.shape, df_lb804.shape

((5, 700), (5, 601))

In [8]:
len( set(df_lb800.columns) & set( df_lb804.columns) )

459

In [9]:
len( set(df_lb804.columns) - set( df_lb803.columns) )

100

In [10]:
f_804_m_803 = list( set(df_lb804.columns) - set( df_lb803.columns) )

In [17]:
f_804_m_803

['f302_notdelay_days_weighted_delay_sum',
 'f505_Closed_DAYS_CREDIT_UPDATE-m-DAYS_ENDDATE_FACT_mean',
 'f701_all_credit_std-dby-income',
 'f501_AMT_CREDIT_SUM_OVERDUE-d-app_AMT_GOODS_PRICE_max',
 'f505_Closed_DAYS_CREDIT_ENDDATE-m-DAYS_CREDIT_mean',
 'f501_CREDIT_TYPE_Consumer-credit_mean',
 'f305_notdelay_con_AMT_PAYMENT-d-app_AMT_INCOME_TOTAL_mean',
 'f308_cons_NUM_INSTALMENT_NUMBER_pctchange_var',
 'f105_future_payment_21m_min',
 'f405_AMT_INST_MIN_REGULARITY_diff_diff_mean',
 'f308_delay_cas_days_weighted_delay_tsw3_diff_var',
 'f101_approved_DAYS_TERMINATION-d-app_DAYS_ID_PUBLISH_min',
 'f308_cons_DAYS_INSTALMENT_pctchange_mean',
 'f001_AMT_CREDIT-d-CNT_CHILDREN',
 'f303_AMT_PAYMENT-d-app_AMT_INCOME_TOTAL_max',
 'f407_AMT_DRAWINGS_OTHER_CURRENT_diff_mean',
 'f313_amt_ratio_pctchange_sum',
 'f501_AMT_CREDIT_SUM_max',
 'f109_AMT_ANNUITY-d-app_AMT_INCOME_TOTAL',
 'f507_Active_DAYS_CREDIT-m-app_DAYS_LAST_PHONE_CHANGE_sum',
 'f001_DAYS_REGISTRATION-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE

In [12]:
len( set(df_lb800.columns) & set(df_lb803.columns) )

401

In [13]:
len( set(df_lb804.columns) & set(df_lb803.columns) )

501

In [16]:
list( set(df_lb804.columns) - set(df_lb800.columns) )

['f302_notdelay_days_weighted_delay_sum',
 'f301_notdelay_AMT_PAYMENT-d-AMT_ANNUITY_mean',
 'f509_DAYS_ENDDATE_FACT-m-app_DAYS_REGISTRATION',
 'f505_Closed_DAYS_CREDIT_UPDATE-m-DAYS_ENDDATE_FACT_mean',
 'f701_all_credit_std-dby-income',
 'f505_Closed_DAYS_CREDIT_ENDDATE-m-DAYS_CREDIT_mean',
 'f101_approved_DAYS_TERMINATION-d-app_DAYS_REGISTRATION_min',
 'f509_DAYS_CREDIT_ENDDATE-m-DAYS_CREDIT',
 'f510_AMT_CREDIT_SUM-d-app_AMT_CREDIT',
 'f308_cons_NUM_INSTALMENT_NUMBER_pctchange_var',
 'f509_DAYS_CREDIT_ENDDATE-m-app_DAYS_BIRTH',
 'f308_delay_cons_DPD_diff_var',
 'f101_approved_DAYS_FIRST_DUE-d-app_DAYS_ID_PUBLISH_min',
 'f105_future_payment_21m_min',
 'f510_AMT_CREDIT_SUM-d-app_AMT_ANNUITY',
 'f106_NAME_PRODUCT_TYPE-XNA_appref',
 'f405_AMT_INST_MIN_REGULARITY_diff_diff_mean',
 'f308_delay_cas_days_weighted_delay_tsw3_diff_var',
 'f510_AMT_CREDIT_SUM',
 'f509_AMT_CREDIT_SUM_DEBT-d-AMT_CREDIT_SUM',
 'f503_DAYS_CREDIT_ENDDATE-m-DAYS_CREDIT_var',
 'f507_Active_DAYS_CREDIT-m-app_DAYS_BIRTH_

In [18]:
[c for c in df_lb802.columns if '_ID_' in c]

['f001_DAYS_ID_PUBLISH-m-DAYS_BIRTH',
 'f505_Active_DAYS_CREDIT-m-app_DAYS_ID_PUBLISH_sum',
 'f001_DAYS_ID_PUBLISH-m-DAYS_REGISTRATION-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_ID_PUBLISH',
 'f001_DAYS_LAST_PHONE_CHANGE-m-DAYS_EMPLOYED-d-DAYS_ID_PUBLISH-m-DAYS_REGISTRATION',
 'f505_Active_DAYS_CREDIT-d-app_DAYS_ID_PUBLISH_min',
 'f001_DAYS_REGISTRATION-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_EMPLOYED',
 'f001_DAYS_ID_PUBLISH-m-DAYS_EMPLOYED-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION',
 'f101_completed_DAYS_LAST_DUE-d-app_DAYS_ID_PUBLISH_var',
 'f101_completed_DAYS_FIRST_DUE-d-app_DAYS_ID_PUBLISH_min',
 'f506_DAYS_CREDIT-m-app_DAYS_ID_PUBLISH_sum',
 'f505_Active_DAYS_CREDIT_ENDDATE-m-app_DAYS_ID_PUBLISH_sum',
 'f505_Active_DAYS_CREDIT-d-app_DAYS_ID_PUBLISH_var',
 'f001_DAYS_ID_PUBLISH-m-DAYS_EMPLOYED-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_ID_PUBLISH',
 'f001_DAYS_REGISTRATION-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_ID_PUBLISH',
 'f001_DAYS_ID_PUBLISH-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_REGISTRATI

# only LB .800 ( this might cause overfitting )

In [9]:
df_lb800.columns.difference(df_lb804.columns).tolist()

['f001_AMT_ANNUITY-d-AMT_INCOME_TOTAL',
 'f001_AMT_CREDIT-d-cnt_adults',
 'f001_AMT_GOODS_PRICE-d-cnt_adults',
 'f001_DAYS_EMPLOYED-d-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_BIRTH',
 'f001_DAYS_ID_PUBLISH-d-DAYS_EMPLOYED',
 'f001_DAYS_ID_PUBLISH-d-DAYS_REGISTRATION',
 'f001_DAYS_LAST_PHONE_CHANGE-d-DAYS_ID_PUBLISH',
 'f001_DAYS_LAST_PHONE_CHANGE-m-DAYS_EMPLOYED-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION',
 'f001_DAYS_REGISTRATION-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_EMPLOYED',
 'f001_DAYS_REGISTRATION-m-DAYS_EMPLOYED',
 'f001_ENTRANCES_AVG',
 'f001_LIVINGAREA_AVG',
 'f001_LIVINGAREA_MODE',
 'f101_active_AMT_ANNUITY-d-app_AMT_ANNUITY_max',
 'f101_approved_AMT_DOWN_PAYMENT_max',
 'f101_approved_AMT_GOODS_PRICE-d-AMT_CREDIT_max',
 'f101_approved_AMT_GOODS_PRICE-d-AMT_CREDIT_mean',
 'f101_approved_CNT_PAYMENT_mean',
 'f101_approved_DAYS_DECISION_max',
 'f101_approved_DAYS_TERMINATIO

In [10]:
feature_ovf = df_lb800.columns.difference(df_lb804.columns).tolist()

In [11]:
len(feature_ovf)

241

# only LB .804 ( this is good features )

In [6]:
df_lb804.columns.difference(df_lb800.columns).tolist()

['f001_AMT_CREDIT-d-CNT_CHILDREN',
 'f001_DAYS_ID_PUBLISH-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_EMPLOYED',
 'f001_DAYS_ID_PUBLISH-m-DAYS_EMPLOYED-d-DAYS_ID_PUBLISH-m-DAYS_REGISTRATION',
 'f001_DAYS_LAST_PHONE_CHANGE-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_EMPLOYED',
 'f001_building_score_mode_sum',
 'f101_approved_AMT_ANNUITY-m-app_AMT_ANNUITY_max',
 'f101_approved_AMT_GOODS_PRICE-d-AMT_CREDIT_var',
 'f101_approved_DAYS_FIRST_DRAWING-d-app_DAYS_BIRTH_mean',
 'f101_approved_DAYS_FIRST_DUE-d-app_DAYS_ID_PUBLISH_min',
 'f101_approved_DAYS_LAST_DUE-d-app_DAYS_BIRTH_mean',
 'f101_approved_DAYS_LAST_DUE-d-app_DAYS_ID_PUBLISH_min',
 'f101_approved_DAYS_LAST_DUE_1ST_VERSION-d-app_DAYS_BIRTH_var',
 'f101_approved_DAYS_TERMINATION-d-app_DAYS_REGISTRATION_min',
 'f101_approved_total_debt_var',
 'f101_completed_AMT_ANNUITY-d-app_AMT_INCOME_TOTAL_max',
 'f101_completed_AMT_ANNUITY-d-app_AMT_INCOME_TOTAL_mean',
 'f101_completed_AMT_ANNUITY-m-app_AMT_ANNUITY-d-app_AMT_INCOME_TOTAL_max',
 'f101_c

In [12]:
only804 = df_lb804.columns.difference(df_lb800.columns).tolist()

In [13]:
len(only804)

142

In [15]:
set(df_lb802.columns) & set(only804)

{'f001_building_score_mode_sum',
 'f101_approved_AMT_ANNUITY-m-app_AMT_ANNUITY_max',
 'f101_completed_AMT_ANNUITY-m-app_AMT_ANNUITY-d-app_AMT_INCOME_TOTAL_max',
 'f101_completed_AMT_APPLICATION-d-app_AMT_CREDIT_max',
 'f101_completed_AMT_GOODS_PRICE-d-total_debt_max',
 'f101_nyg-high_AMT_DOWN_PAYMENT_max',
 'f101_nyg-middle_AMT_GOODS_PRICE-d-AMT_CREDIT_min',
 'f103_nyg-low_normal_AMT_ANNUITY-m-app_AMT_INCOME_TOTAL_max',
 'f105_approved_ratio',
 'f105_future_payment_21m_min',
 'f109_AMT_ANNUITY-d-app_AMT_INCOME_TOTAL',
 'f301_AMT_PAYMENT-d-app_AMT_ANNUITY_mean',
 'f301_DAYS_ENTRY_PAYMENT-m-app_DAYS_LAST_PHONE_CHANGE_min',
 'f301_DAYS_ENTRY_PAYMENT_max',
 'f301_notdelay_AMT_PAYMENT-d-AMT_ANNUITY_mean',
 'f302_DAYS_ENTRY_PAYMENT_mean',
 'f302_days_weighted_delay_sum',
 'f302_notdelay_AMT_PAYMENT-d-app_AMT_ANNUITY_min',
 'f302_notdelay_NUM_INSTALMENT_ratio_max',
 'f302_notdelay_days_weighted_delay_sum',
 'f303_DPD_max',
 'f305_con_AMT_PAYMENT_min',
 'f305_delay_cas_DAYS_ENTRY_PAYMENT_sum',

# only LB .800 from .802 ( this might cause overfitting )

In [9]:
df_lb800.columns.difference(df_lb802.columns).tolist()

['f001_AMT_ANNUITY-d-CNT_CHILDREN',
 'f001_AMT_CREDIT-d-cnt_adults',
 'f001_AMT_GOODS_PRICE-d-cnt_adults',
 'f001_AMT_INCOME_TOTAL-d-CNT_FAM_MEMBERS',
 'f001_AMT_REQ_CREDIT_BUREAU_QRT',
 'f001_APARTMENTS_AVG',
 'f001_DAYS_EMPLOYED-d-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_EMPLOYED',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_REGISTRATION',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_ID_PUBLISH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION',
 'f001_DAYS_ID_PUBLISH-d-DAYS_EMPLOYED',
 'f001_DAYS_ID_PUBLISH-d-DAYS_REGISTRATION',
 'f001_DAYS_ID_PUBLISH-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION',
 'f001_DAYS_ID_PUBLISH-m-DAYS_BIRTH-d-DAYS_REGISTRATION-m-DAYS_EMPLOYED',
 'f001_DAYS_ID_PUBLISH-m-DAYS_EMPLOYED-d-DAYS_LAST_PHONE_CHANGE-m-

In [10]:
set(df_lb800.columns.difference(df_lb804.columns).tolist()) & set( df_lb800.columns.difference(df_lb802.columns).tolist() )

{'f001_AMT_CREDIT-d-cnt_adults',
 'f001_AMT_GOODS_PRICE-d-cnt_adults',
 'f001_DAYS_EMPLOYED-d-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_ID_PUBLISH-m-DAYS_BIRTH',
 'f001_DAYS_EMPLOYED-m-DAYS_BIRTH-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_BIRTH',
 'f001_DAYS_ID_PUBLISH-d-DAYS_EMPLOYED',
 'f001_DAYS_ID_PUBLISH-d-DAYS_REGISTRATION',
 'f001_DAYS_LAST_PHONE_CHANGE-d-DAYS_ID_PUBLISH',
 'f001_DAYS_LAST_PHONE_CHANGE-m-DAYS_EMPLOYED-d-DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION',
 'f001_DAYS_REGISTRATION-m-DAYS_EMPLOYED',
 'f001_ENTRANCES_AVG',
 'f001_LIVINGAREA_MODE',
 'f101_active_AMT_ANNUITY-d-app_AMT_ANNUITY_max',
 'f101_approved_AMT_GOODS_PRICE-d-AMT_CREDIT_max',
 'f101_approved_AMT_GOODS_PRICE-d-AMT_CREDIT_mean',
 'f101_approved_DAYS_DECISION_max',
 'f101_approved_DAYS_TERMINATION-d-app_DAYS_EMPLOYED_min',
 'f101_approved_cnt_paid_var',
 'f101_completed_AMT_GOODS_PRICE-d-total_debt_mean',
 'f101_completed_DAYS_FIRST_DUE-d-app_DAYS_BIRTH_min',
 'f101_completed_DAYS_LAST_DUE_1ST_VERSION-d-

# load LB 800 and 804

In [5]:
files = ('../feature/train_' + df_lb800.columns + '.f').tolist()
X_train_lb800 = pd.concat([
                pd.read_feather(f) for f in tqdm(files, mininterval=60)
               ], axis=1)


100%|██████████| 700/700 [01:04<00:00, 10.78it/s]


In [6]:
files = ('../feature/test_' + df_lb800.columns + '.f').tolist()
X_test_lb800 = pd.concat([
                pd.read_feather(f) for f in tqdm(files, mininterval=60)
               ], axis=1)

100%|██████████| 700/700 [00:28<00:00, 24.30it/s]


In [7]:
files = ('../feature/train_' + df_lb804.columns + '.f').tolist()
X_train_lb804 = pd.concat([
                pd.read_feather(f) for f in tqdm(files, mininterval=60)
               ], axis=1)


100%|██████████| 601/601 [00:15<00:00, 39.07it/s]


In [8]:
files = ('../feature/test_' + df_lb804.columns + '.f').tolist()
X_test_lb804 = pd.concat([
                pd.read_feather(f) for f in tqdm(files, mininterval=60)
               ], axis=1)

100%|██████████| 601/601 [00:06<00:00, 96.20it/s]


In [9]:
y = utils.read_pickles('../data/label').TARGET

100%|██████████| 20/20 [00:00<00:00, 88.49it/s]


# CV

In [10]:
param = {
         'objective': 'binary',
         'metric': 'auc',
         'learning_rate': 0.01,
         'max_depth': 6,
         'num_leaves': 63,
         'max_bin': 255,
         
         'min_child_weight': 10,
         'min_data_in_leaf': 150,
         'reg_lambda': 0.5,  # L2 regularization term on weights.
         'reg_alpha': 0.5,  # L1 regularization term on weights.
         
         'colsample_bytree': 0.9,
         'subsample': 0.9,
#         'nthread': 32,
         'nthread': cpu_count(),
         'bagging_freq': 1,
         'verbose':-1,
         'seed': 71
         }


In [24]:
CAT = list( set(X_train_lb800.columns)&set(utils_cat.ALL))
dtrain = lgb.Dataset(X_train_lb800, y, categorical_feature=CAT )
gc.collect()

ret = lgb.cv(param, dtrain, 9999, nfold=7,
             early_stopping_rounds=100, verbose_eval=50,
             seed=71)

result = f"CV auc-mean: {ret['auc-mean'][-1]} + {ret['auc-stdv'][-1]}"
print(result)


[50]	cv_agg's auc: 0.769823 + 0.00338049
[100]	cv_agg's auc: 0.773407 + 0.00323751
[150]	cv_agg's auc: 0.776822 + 0.00322845
[200]	cv_agg's auc: 0.780142 + 0.00328529
[250]	cv_agg's auc: 0.783127 + 0.00331152
[300]	cv_agg's auc: 0.785781 + 0.00323746
[350]	cv_agg's auc: 0.787881 + 0.00322167
[400]	cv_agg's auc: 0.789557 + 0.00311069
[450]	cv_agg's auc: 0.79099 + 0.003146
[500]	cv_agg's auc: 0.792149 + 0.00310973
[550]	cv_agg's auc: 0.793187 + 0.0031717
[600]	cv_agg's auc: 0.794078 + 0.00321107
[650]	cv_agg's auc: 0.794784 + 0.00319545
[700]	cv_agg's auc: 0.795468 + 0.0032306
[750]	cv_agg's auc: 0.795997 + 0.00329536
[800]	cv_agg's auc: 0.796494 + 0.0032991
[850]	cv_agg's auc: 0.796986 + 0.00332342
[900]	cv_agg's auc: 0.797418 + 0.00334438
[950]	cv_agg's auc: 0.797817 + 0.00336117
[1000]	cv_agg's auc: 0.798161 + 0.00335759
[1050]	cv_agg's auc: 0.798496 + 0.00333883
[1100]	cv_agg's auc: 0.798796 + 0.0033446
[1150]	cv_agg's auc: 0.79912 + 0.00335258
[1200]	cv_agg's auc: 0.799391 + 0.00334

KeyboardInterrupt: 

In [25]:
CAT = list( set(X_train_lb800.columns)&set(utils_cat.ALL))
dtrain = lgb.Dataset(X_train_lb800, y, categorical_feature=CAT )
gc.collect()

model = lgb.train(param, dtrain, 3000)
imp_lb800 = ex.getImp(model).sort_values(['gain', 'feature'], ascending=[False, True])


In [26]:
CAT = list( set(X_train_lb804.columns)&set(utils_cat.ALL))
dtrain = lgb.Dataset(X_train_lb804, y, categorical_feature=CAT )
gc.collect()

model = lgb.train(param, dtrain, 3000)
imp_lb804 = ex.getImp(model).sort_values(['gain', 'feature'], ascending=[False, True])



In [27]:
imp_lb800.to_csv('lb800.csv', index=False)
imp_lb804.to_csv('lb804.csv', index=False)


In [30]:
imp_lb800_ovf = imp_lb800[imp_lb800.feature.isin(feature_ovf)]

In [31]:
imp_lb800_ovf.to_csv('lb800_ovf.csv', index=False)

# adversarial validation with ovf

In [11]:
X_train_lb800_ovf = X_train_lb800[feature_ovf]
X_train_lb800_ovf['target'] = 1

X_test_lb800_ovf  = X_test_lb800[feature_ovf]
X_test_lb800_ovf['target'] = 0

X_ovf = pd.concat([X_train_lb800_ovf, X_test_lb800_ovf], ignore_index=True)
y_ovf = X_ovf.target
X_ovf.drop('target', axis=1, inplace=True)

In [ ]:
CAT = list( set(X_ovf.columns)&set(utils_cat.ALL))
dtrain = lgb.Dataset(X_ovf, y_ovf, categorical_feature=CAT )
gc.collect()

ret = lgb.cv(param, dtrain, 9999, nfold=7,
             early_stopping_rounds=100, verbose_eval=50,
             seed=71)

result = f"CV auc-mean: {ret['auc-mean'][-1]} + {ret['auc-stdv'][-1]}"
print(result)

[50]	cv_agg's auc: 0.880407 + 0.00124874
[100]	cv_agg's auc: 0.8837 + 0.00121335
[150]	cv_agg's auc: 0.886695 + 0.00132782
[200]	cv_agg's auc: 0.889878 + 0.00120145
[250]	cv_agg's auc: 0.892953 + 0.00131629
[300]	cv_agg's auc: 0.89584 + 0.00130764
[350]	cv_agg's auc: 0.898574 + 0.00127852
[400]	cv_agg's auc: 0.90172 + 0.00122743
[450]	cv_agg's auc: 0.903881 + 0.00125811
[500]	cv_agg's auc: 0.905632 + 0.00122126
[550]	cv_agg's auc: 0.907056 + 0.00125124
[600]	cv_agg's auc: 0.908316 + 0.00124777
[650]	cv_agg's auc: 0.909413 + 0.00120961
[700]	cv_agg's auc: 0.910371 + 0.00122499
[750]	cv_agg's auc: 0.911307 + 0.00124072
[800]	cv_agg's auc: 0.912172 + 0.0012671
[850]	cv_agg's auc: 0.913 + 0.00123488
[900]	cv_agg's auc: 0.913781 + 0.00123768
[950]	cv_agg's auc: 0.914547 + 0.00121634
[1000]	cv_agg's auc: 0.91523 + 0.00123803
[1050]	cv_agg's auc: 0.915936 + 0.00125653
[1100]	cv_agg's auc: 0.916571 + 0.00124066
[1150]	cv_agg's auc: 0.917216 + 0.00124012
[1200]	cv_agg's auc: 0.917806 + 0.001271

In [12]:
CAT = list( set(X_ovf.columns)&set(utils_cat.ALL))
dtrain = lgb.Dataset(X_ovf, y_ovf, categorical_feature=CAT )
gc.collect()

model = lgb.train(param, dtrain, 1000)
imp_ovf = ex.getImp(model).sort_values(['gain', 'feature'], ascending=[False, True])


In [13]:
imp_ovf

,feature,split,gain
0,f601_Closed_MONTHS_BALANCE_sum,3117,2.197206e+06
1,f602_Active_MONTHS_BALANCE_mean,2134,6.617591e+05
2,f505_Closed_AMT_CREDIT_SUM-d-app_AMT_INCOME_TO...,227,5.399114e+05
3,f505_Active_DAYS_CREDIT-d-app_DAYS_BIRTH_min,307,1.641696e+05
4,f505_Active_DAYS_CREDIT_max,872,1.605563e+05
5,f105_prevapp_future_payment_21m,915,1.570887e+05
6,f505_Closed_DAYS_ENDDATE_FACT-m-DAYS_CREDIT_sum,1196,1.431801e+05
7,f501_DAYS_CREDIT_min,1387,1.410532e+05
8,f105_prevapp_future_payment_20m,597,8.742117e+04
9,f505_Active_DAYS_CREDIT_UPDATE_var,876,6.358519e+04
